# ETL phase 3: Data Cleansing
<img src="data-prep-kdd-process-crisp-dm.jpg" width="1000" height="600">
<h2><center>Data preparation in both the KDD Process (left) and the CRISP-DM model (right).</center></h2>
<img src="kdd.gif" width="800" height="500">
<h2><center>KDD - Knowledge Discovery in Databases</center></h2>
<img src="CRISP-DM_Process_Diagram.png" width="500" height="300">
<h2><center>CRISP - Cross-industry standard process for data mining</center></h2>

Data cleansing consists of following 3 processes usally<br> 
* Missing Values
* Outlier Values
* Duplidated values

## 1. import necessary modules（Optional）

In [32]:
#==============================================
#=== Mothod 1: Install and import necessary modules
#==============================================
from tool_import_modules import *
modules = ['os', 'pandas', 'pyodbc', 'numpy', 'glob', 'seaborn', 'matplotlib', 'logging', 'time', 'openpyxl', 'dask']
for module in modules:
    import_neccessary_modules(module)

In [33]:
#==============================================
#=== Mothod 2: Import modules directly
#==============================================
import os
import pandas
import pyodbc
import numpy as np
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import logging
import time
import openpyxl as xl
from openpyxl import Workbook
from openpyxl import load_workbook
import dask.dataframe as dd

## 2. Set path, config, and connection（Optional）

In [34]:
# Set path
my_dbName = '0179Orders_Org'
my_path = r"C:\MyDataFiles\Data_JayCoop_202109"
my_path_DB = my_path + "\DB"
my_path_DW = my_path + "\DW"
my_path_cleaned = my_path + "\cleaned"
directors =  [my_path_DB, my_path_DW, my_path_cleaned]
# Set file names
log_fileName = time.strftime("%Y%m%d") + '_DB.log'
audit_fileName = time.strftime("%Y%m%d") + '_DB_audit.xlsx'
audit_fullPath = os.path.join(my_path, audit_fileName)
# Set log file
os.chdir(my_path)
logger = logging.getLogger()
while logger.hasHandlers():
    logger.removeHandler(logger.handlers[0])
#logger.setLevel(logging.DEBUG)
# Create file handler which logs even debug messages
fh = logging.FileHandler(log_fileName, 'w') # 'w'-overwrite; 'a'-append
fh.setLevel(logging.INFO)
# Create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
# Create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s : [%(levelname)s] %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)
# Add the handlers to the logger
logger.addHandler(fh)
logger.addHandler(ch)
# Check path
if not os.path.exists(my_path):
    os.makedirs(my_path)
    logger.info("Directory created: " + my_path)
# Check directors
for director in directors:
    if not os.path.exists(director):
        os.makedirs(director)
        logger.debug('\nDirectory created: ' + director)
# Check auditExcel
if not os.path.isfile(audit_fullPath):
    auditExcel = Workbook()
    sheet1 = auditExcel.active
    sheet1.title = 'CreatedFiles'
    sheet1.append(["File", "CreatedTime", "Path"])
    sheet2 = auditExcel.create_sheet(title="Cleansing")
    sheet2.append(["Database", "Table", "Column", "Value", "Issue"])
    auditExcel.save(audit_fullPath)
else:
    auditExcel = load_workbook(filename = audit_fullPath)
    sheet1 = auditExcel["CreatedFiles"]
    sheet2 = auditExcel["Cleansing"]
    adtExcSh1Row = sheet1.max_row
    adtExcSh2Row = sheet1.max_row
# Print log header
logger.info('==== Cleaning Start ====')
# Set up SQL Server connector (DATABASE:'0179Orders_Org')
os.chdir(my_path_DB)
sql_conn = pyodbc.connect('DRIVER={SQL Server}; SERVER=localhost; DATABASE=0179Orders_Org; UID=sa; PWD=SQLServer2019')

2021-09-28 14:32:01,856 : [INFO] ==== Cleaning Start ====


## 3. Get data

In [38]:
# Get table name list
os.chdir(my_path_DB)
db_files=os.listdir()
chunksize = 100000
print(db_files)
createVar = locals()
for i in db_files:
    if i.endswith("csv"):
        tableName = i.split('.')[0]
        createVar[tableName] = pandas.read_csv(i, chunksize=chunksize, dtype={'PostalCode': 'object', 'Notes': 'object', 'Photo': 'object', 'PhotoPath': 'object'})
        #df = pandas.concat(createVar[tableName])
        #print(createVar[tableName])
        

['Categories.csv', 'CustomerCustomerDemo.csv', 'CustomerDemographics.csv', 'Customers.csv', 'Employees.csv', 'EmployeeTerritories.csv', 'Order Details.csv', 'Orders.csv', 'Products.csv', 'Region.csv', 'Shippers.csv', 'Suppliers.csv', 'Territories.csv', 'test.csv']


## 4. Data overview

In [40]:
# dataframe Categories
df = pandas.concat(Categories)
Categories.info()

AttributeError: 'TextFileReader' object has no attribute 'info'

In [8]:
# Cleaning
os.chdir(my_path_DB)
#colours = ['#000099', '#ffff00'] # specify the colours - yellow is missing. blue is not missing.
logger.info("==== Cleaning Start ====")
for file in glob.glob("*.csv"):
    tableName = str(file)[:-4]    
    cleanFile = tableName + "_clean.csv"   

    # Get PK
    pkNameQuery = "SELECT Col.Column_Name as PkName from INFORMATION_SCHEMA.TABLE_CONSTRAINTS Tab, INFORMATION_SCHEMA.CONSTRAINT_COLUMN_USAGE Col WHERE Col.Constraint_Name = Tab.Constraint_Name AND Col.Table_Name = Tab.Table_Name AND Constraint_Type = 'PRIMARY KEY' AND Col.Table_Name = '" + tableName +"'"
    pkList = list(pandas.read_sql(pkNameQuery, sql_conn)["PkName"])

    # Get data
    df = pandas.read_csv(file, index_col = pkList)
    size_org = df.shape[0]
    cols = df.columns
    logger.info('From: ' + file + ' - size' + str(df.shape))

    # Drop duplicate
    df.drop_duplicates(keep="first", inplace=True)

    # Print duplication info
    size_cleaned = df.shape[0]
    logger.info('To    : ' + cleanFile + ' - size' + str(df.shape))
    num_duplication = size_org - size_cleaned
    if num_duplication > 0:
        logger.info('------ [Duplication] ' + str(num_duplication) + ' records dropped from ' + file)
        adtExcSh2Row = adtExcSh2Row + 1
        sheet2.cell(row=adtExcSh2Row, column=1).value = 'Database'
        sheet2.cell(row=adtExcSh2Row, column=2).value = tableName
        sheet2.cell(row=adtExcSh2Row, column=3).value = '-'
        sheet2.cell(row=adtExcSh2Row, column=4).value = int(num_duplication)
        sheet2.cell(row=adtExcSh2Row, column=5).value = 'Duplication' 
    # Set numeric columns
    df_numeric = df.select_dtypes(include=[np.number])
    numeric_cols = df_numeric.columns.values
   
    # Set non numeric columns
    df_non_numeric = df.select_dtypes(exclude=[np.number])
    non_numeric_cols = df_non_numeric.columns.values

    for col in df.columns:
        # cleaning missing
        missing = df[col].isnull()
        num_missing = np.sum(missing)
        pct_missing = np.mean(missing)
             
        if num_missing > 0: 

            # Print Missing Data Percentage List - % of missing.
            df['{}_ismissing'.format(col)] = missing


            # When numeric, fill with midian value 
            if col in numeric_cols:
                med = df[col].median()
                if col == 'NPS':
                    med = int(med)
                df[col] = df[col].fillna(med)             
                logger.info('------ [Missing] ' + file + ' - "{}" - {}%'.format(col, round(pct_missing*100)) + ', ' + str(num_missing) + ' records missed - filling with ' + str(med))
            # When not numeric, fill with most frequent value     
            else:
                top = df[col].describe()['top'] # impute with the most frequent value.
                df[col] = df[col].fillna(top)
                logger.info('------ [Missing] ' + file + ' - "{}" - {}%'.format(col, round(pct_missing*100)) + ', ' + str(num_missing) + ' records missed - filling with "' + top + '"')
            
            adtExcSh2Row = adtExcSh2Row + 1
            sheet2.cell(row=adtExcSh2Row, column=1).value = my_dbName
            sheet2.cell(row=adtExcSh2Row, column=2).value = tableName
            sheet2.cell(row=adtExcSh2Row, column=3).value = col
            sheet2.cell(row=adtExcSh2Row, column=4).value = int(num_missing)
            sheet2.cell(row=adtExcSh2Row, column=5).value = 'Missing'  
        # cleaning outliner
        #df.boxplot(column=col)

    # write to the new csf in 'cleaned' director
    try:
        df.to_csv(my_path_cleaned + "/" + cleanFile)
        adtExcSh1Row = adtExcSh1Row + 1
        sheet1.cell(row=adtExcSh1Row, column=1).value = str(cleanFile)
        sheet1.cell(row=adtExcSh1Row, column=2).value = time.asctime()
        sheet1.cell(row=adtExcSh1Row, column=3).value = my_path_cleaned
    except:
        tb = sys.exc_info()[2]
        logger.warn('**** File did NOT update successfully. Please try again after make sure file is not opened and have pomission to write. - ' + my_path_cleaned + "/" + cleanFile)
        continue
    
    os.chdir(my_path_DB)

auditExcel.save(audit_fullPath)
logger.info('Cleaning Completed Successfully - ' + str(len(os.listdir(my_path_cleaned))) + ' files created in ' + my_path_cleaned)    

2021-09-28 00:49:11,552 : [INFO] ==== Cleaning Start ====
2021-09-28 00:49:11,916 : [INFO] From: Categories.csv - size(1009, 3)
2021-09-28 00:49:11,923 : [INFO] To    : Categories_clean.csv - size(906, 3)
2021-09-28 00:49:11,925 : [INFO] ------ [Duplication] 103 records dropped from Categories.csv
2021-09-28 00:49:12,058 : [INFO] ------ [Missing] Categories.csv - "Description" - 99%, 897 records missed - filling with "Soft drinks, coffees, teas, beers, and ales"
2021-09-28 00:49:12,063 : [INFO] ------ [Missing] Categories.csv - "Picture" - 99%, 898 records missed - filling with "b"\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\x14\x00!\x00\xff\xff\xff\xffBitmap Image\x00Paint.Picture\x00\x01\x05\x00\x00\x02\x00\x00\x00\x07\x00\x00\x00PBrush\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa0)\x00\x00BM\x98)\x00\x00\x00\x00\x00\x00V\x00\x00\x00(\x00\x00\x00\xac\x00\x00\x00x\x00\x00\x00\x01\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x88\x0b\x00\x00\x88\x0b\x00\x00\x08\x00\x00\x00\x08\x00\x00\x00\xff\x

ParserError: Error tokenizing data. C error: out of memory